# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
#print(data['symbol'])
#print(data.status_code)
print(data)

{'companyName': 'Apple Inc', 'marketcap': 2298354241731, 'week52high': 141.61, 'week52low': 57.21, 'week52change': 0.812308273299656, 'sharesOutstanding': 17401967831, 'float': 0, 'avg10Volume': 99227885, 'avg30Volume': 107075344, 'day200MovingAvg': 116.75, 'day50MovingAvg': 128.23, 'employees': 0, 'ttmEPS': 3.4, 'ttmDividendRate': 0.8112194077475431, 'dividendYield': 0.006146379185990386, 'nextDividendDate': '0', 'exDividendDate': '2020-11-01', 'nextEarningsDate': '0', 'peRatio': 40.14200423839566, 'beta': 1.1824328465665734, 'maxChangePercent': 50.57070029931097, 'year5ChangePercent': 4.477329168603243, 'year2ChangePercent': 2.4767082546391967, 'year1ChangePercent': 0.8511234284762521, 'ytdChangePercent': 0, 'month6ChangePercent': 0.46321685602201607, 'month3ChangePercent': 0.148002778736371, 'month1ChangePercent': 0.1154731869531669, 'day30ChangePercent': 0.08175903284834962, 'day5ChangePercent': 0.00565657018524217}


## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.8511234284762521

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe=pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy


In [7]:
final_dataframe=pd.DataFrame(columns=my_columns)
#final_dataframe
for symbol_string in symbol_strings:
    batch_api_url= f'https://sandbox.iexapis.com/stable/stock/market/batch/?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    #print(batch_api_url)
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        
        
       final_dataframe=final_dataframe.append(
        pd.Series(
            [
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
            ],
            index = my_columns

        ),
            ignore_index = True

        )
   
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,121.76,0.416234,N/A
1,AAL,16.07,-0.460852,N/A
2,AAP,160.99,-0.00898951,N/A
3,AAPL,136.98,0.827153,N/A
4,ABBV,108.41,0.282779,N/A
5,ABC,99.11,0.170499,N/A
6,ABMD,329.30,0.942197,N/A
7,ABT,112.31,0.285734,N/A
8,ACN,268.40,0.256944,N/A
9,ADBE,509.46,0.53793,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [39]:
final_dataframe.sort_values('One-Year Price Return',ascending = False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace= True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,37.330,2.24513,N/A
1,345,NVDA,522.890,1.20888,N/A
2,275,LB,39.710,1.20627,N/A
3,387,PYPL,234.740,1.17102,N/A
4,23,ALB,148.690,1.11849,N/A
5,31,AMD,91.420,0.993661,N/A
6,339,NOW,545.340,0.932792,N/A
7,3,AAPL,136.910,0.931935,N/A
8,85,CDNS,136.550,0.928679,N/A
9,179,FCX,24.730,0.926743,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [10]:

def portfolio_input():
    global Position_Size
    try:
        Position_Size = input('Enter the Value of Position:  ')
        val = float(Position_Size)
    except ValueError:
        print('Please enter the valid number an try again')
        Position_Size = input('Enter the Value of Position:  ')
        val = float(Position_Size)
portfolio_input()
print(Position_Size)

Enter the Value of Position:  10000000
10000000


In [15]:
position_size = float(Position_Size)/len(final_dataframe.index)
#position_size
for i in range(0,len(final_dataframe)):
    final_dataframe.loc[i,'Number of Shares to Buy'] =math.floor(position_size/final_dataframe.loc[i,'Price'])
#final_dataframe

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [55]:
hqm_columns = [
    'ticker',
    'price',
    'number of shares to buy',
    'one-year price return',
    'one-year return percentile',
    'six-month price return',
    'six-month return percentile',
    'three-month price return',
    'three-month return percentile',
    'one-month price return',
    'one-month return percentile',
    'HQM score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)
#hqm_dataframe

for symbol_string in symbol_strings:
    batch_api_url= f'https://sandbox.iexapis.com/stable/stock/market/batch/?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    print(batch_api_url)
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        
        
       hqm_dataframe=hqm_dataframe.append(
        pd.Series(
            [
        symbol,
        data[symbol]['price'],
        'N/A',
        data[symbol]['stats']['year1ChangePercent'],
        'N/A',
        data[symbol]['stats']['month6ChangePercent'],
        'N/A',
        data[symbol]['stats']['month3ChangePercent'],
        'N/A',
        data[symbol]['stats']['month1ChangePercent'],
        'N/A',
        'N/A'
            ],
            index = hqm_columns

        ),
            ignore_index = True

        )
hqm_dataframe

https://sandbox.iexapis.com/stable/stock/market/batch/?symbols=A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA&types=price,stats&token=Tpk_059b97af715d417d9f49f50b51b1c448
https://sandbox.iexapis.com/stable/stock/market/batch/?symbols=CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,

,ticker,price,number of shares to buy,one-year price return,one-year return percentile,six-month price return,six-month return percentile,three-month price return,three-month return percentile,one-month price return,one-month return percentile,HQM score
0,A,120.37,N/A,0.399187,N/A,0.345169,N/A,0.183081,N/A,0.0138943,N/A,N/A
1,AAL,16.00,N/A,-0.451928,N/A,0.215159,N/A,0.287377,N/A,0.118225,N/A,N/A
2,AAP,160.86,N/A,-0.00904387,N/A,0.113559,N/A,0.0290019,N/A,0.0712209,N/A,N/A
3,AAPL,136.83,N/A,0.845506,N/A,0.461787,N/A,0.147892,N/A,0.116041,N/A,N/A
4,ABBV,107.62,N/A,0.28526,N/A,0.120504,N/A,0.249557,N/A,0.025108,N/A,N/A
5,ABC,101.92,N/A,0.172478,N/A,-0.0222505,N/A,0.0128977,N/A,-0.053594,N/A,N/A
6,ABMD,336.30,N/A,0.921331,N/A,0.352269,N/A,0.173244,N/A,0.184745,N/A,N/A
7,ABT,110.99,N/A,0.284975,N/A,0.212681,N/A,0.00946475,N/A,0.0119458,N/A,N/A
8,ACN,270.40,N/A,0.264018,N/A,0.22628,N/A,0.167966,N/A,0.0502328,N/A,N/A
9,ADBE,514.87,N/A,0.536339,N/A,0.155379,N/A,0.0198075,N/A,0.046547,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [80]:
time_periods = [
    
    'one-year',
    'six-month',
    'three-month',
    'one-month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row,f'{time_period} price return'] == None:
            hqm_dataframe.loc[row,f'{time_period} price return'] = 0.0

for row in hqm_dataframe.index:
    for time_period in time_periods:            
        #hqm_dataframe.loc[row,f'{time_period} return percentile']= 0
        hqm_dataframe.loc[row, f'{time_period} return percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} price return'], hqm_dataframe.loc[row, f'{time_period} price return'])/100

hqm_dataframe


,ticker,price,number of shares to buy,one-year price return,one-year return percentile,six-month price return,six-month return percentile,three-month price return,three-month return percentile,one-month price return,one-month return percentile,HQM score
179,FCX,26.34,0,1.00072,0.988119,1.26588,0.99604,0.671136,0.986139,0.117286,0.877228,96.1881
24,ALGN,543.14,0,0.941378,0.982178,0.976398,0.990099,0.663676,0.984158,0.113531,0.869307,95.6436
410,SIVB,405.22,0,0.552967,0.928713,0.804066,0.974257,0.629831,0.974257,0.126926,0.89901,94.4059
23,ALB,153.73,0,1.10508,0.992079,0.933206,0.988119,0.66082,0.980198,0.0885878,0.79604,93.9109
324,MU,77.41,0,0.403482,0.865347,0.463347,0.837624,0.603658,0.970297,0.177012,0.968317,91.0396
134,DIS,189.45,0,0.256797,0.718812,0.645587,0.942574,0.460663,0.924752,0.230364,0.988119,89.3564
45,APTV,134.84,0,0.384832,0.845545,0.683713,0.956436,0.432275,0.90099,0.101422,0.839604,88.5644
358,PAYC,469.87,0,0.723469,0.956436,0.482493,0.863366,0.468883,0.928713,0.0845722,0.778218,88.1683
448,TWTR,54.88,0,0.707,0.952475,0.842999,0.976238,0.224106,0.627723,0.168556,0.964356,88.0198
28,ALXN,158.53,0,0.450409,0.891089,0.400937,0.754455,0.382325,0.867327,0.292647,0.99802,87.7723


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [82]:
#hqm_dataframe['one-year price return'].describe()
from statistics import mean
#mean([5,2])

for row in hqm_dataframe.index:
    momemtum_percentiles = []
    for time_period in time_periods:
        momemtum_percentiles.append(hqm_dataframe.loc[row,f'{time_period} return percentile'])
    hqm_dataframe.loc[row,'HQM score']= mean(momemtum_percentiles)
    #print(momemtum_percentiles)
hqm_dataframe

,ticker,price,number of shares to buy,one-year price return,one-year return percentile,six-month price return,six-month return percentile,three-month price return,three-month return percentile,one-month price return,one-month return percentile,HQM score
179,FCX,26.34,0,1.00072,0.988119,1.26588,0.99604,0.671136,0.986139,0.117286,0.877228,0.961881
24,ALGN,543.14,0,0.941378,0.982178,0.976398,0.990099,0.663676,0.984158,0.113531,0.869307,0.956436
410,SIVB,405.22,0,0.552967,0.928713,0.804066,0.974257,0.629831,0.974257,0.126926,0.89901,0.944059
23,ALB,153.73,0,1.10508,0.992079,0.933206,0.988119,0.66082,0.980198,0.0885878,0.79604,0.939109
324,MU,77.41,0,0.403482,0.865347,0.463347,0.837624,0.603658,0.970297,0.177012,0.968317,0.910396
134,DIS,189.45,0,0.256797,0.718812,0.645587,0.942574,0.460663,0.924752,0.230364,0.988119,0.893564
45,APTV,134.84,0,0.384832,0.845545,0.683713,0.956436,0.432275,0.90099,0.101422,0.839604,0.885644
358,PAYC,469.87,0,0.723469,0.956436,0.482493,0.863366,0.468883,0.928713,0.0845722,0.778218,0.881683
448,TWTR,54.88,0,0.707,0.952475,0.842999,0.976238,0.224106,0.627723,0.168556,0.964356,0.880198
28,ALXN,158.53,0,0.450409,0.891089,0.400937,0.754455,0.382325,0.867327,0.292647,0.99802,0.877723


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [83]:
hqm_dataframe.sort_values('HQM score',ascending = False, inplace=True)
hqm_dataframe_top50 = hqm_dataframe[:50]
hqm_dataframe_top50.reset_index(drop = True, inplace= True)
hqm_dataframe_top50


,ticker,price,number of shares to buy,one-year price return,one-year return percentile,six-month price return,six-month return percentile,three-month price return,three-month return percentile,one-month price return,one-month return percentile,HQM score
0,FCX,26.34,0,1.00072,0.988119,1.26588,0.99604,0.671136,0.986139,0.117286,0.877228,0.961881
1,ALGN,543.14,0,0.941378,0.982178,0.976398,0.990099,0.663676,0.984158,0.113531,0.869307,0.956436
2,SIVB,405.22,0,0.552967,0.928713,0.804066,0.974257,0.629831,0.974257,0.126926,0.89901,0.944059
3,ALB,153.73,0,1.10508,0.992079,0.933206,0.988119,0.66082,0.980198,0.0885878,0.79604,0.939109
4,MU,77.41,0,0.403482,0.865347,0.463347,0.837624,0.603658,0.970297,0.177012,0.968317,0.910396
5,DIS,189.45,0,0.256797,0.718812,0.645587,0.942574,0.460663,0.924752,0.230364,0.988119,0.893564
6,APTV,134.84,0,0.384832,0.845545,0.683713,0.956436,0.432275,0.90099,0.101422,0.839604,0.885644
7,PAYC,469.87,0,0.723469,0.956436,0.482493,0.863366,0.468883,0.928713,0.0845722,0.778218,0.881683
8,TWTR,54.88,0,0.707,0.952475,0.842999,0.976238,0.224106,0.627723,0.168556,0.964356,0.880198
9,ALXN,158.53,0,0.450409,0.891089,0.400937,0.754455,0.382325,0.867327,0.292647,0.99802,0.877723


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [84]:
portfolio_input()


Enter the Value of Position:  1000000


In [85]:
position_size = float(Position_Size)/len(hqm_dataframe_top50.index)
position_size

for i in hqm_dataframe_top50.index:
    hqm_dataframe_top50.loc[i,'number of shares to buy'] = math.floor(position_size/hqm_dataframe_top50.loc[i,'price'])
hqm_dataframe_top50


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,ticker,price,number of shares to buy,one-year price return,one-year return percentile,six-month price return,six-month return percentile,three-month price return,three-month return percentile,one-month price return,one-month return percentile,HQM score
0,FCX,26.34,759,1.00072,0.988119,1.26588,0.99604,0.671136,0.986139,0.117286,0.877228,0.961881
1,ALGN,543.14,36,0.941378,0.982178,0.976398,0.990099,0.663676,0.984158,0.113531,0.869307,0.956436
2,SIVB,405.22,49,0.552967,0.928713,0.804066,0.974257,0.629831,0.974257,0.126926,0.89901,0.944059
3,ALB,153.73,130,1.10508,0.992079,0.933206,0.988119,0.66082,0.980198,0.0885878,0.79604,0.939109
4,MU,77.41,258,0.403482,0.865347,0.463347,0.837624,0.603658,0.970297,0.177012,0.968317,0.910396
5,DIS,189.45,105,0.256797,0.718812,0.645587,0.942574,0.460663,0.924752,0.230364,0.988119,0.893564
6,APTV,134.84,148,0.384832,0.845545,0.683713,0.956436,0.432275,0.90099,0.101422,0.839604,0.885644
7,PAYC,469.87,42,0.723469,0.956436,0.482493,0.863366,0.468883,0.928713,0.0845722,0.778218,0.881683
8,TWTR,54.88,364,0.707,0.952475,0.842999,0.976238,0.224106,0.627723,0.168556,0.964356,0.880198
9,ALXN,158.53,126,0.450409,0.891089,0.400937,0.754455,0.382325,0.867327,0.292647,0.99802,0.877723


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [86]:
writer = pd.ExcelWriter('Momemtum_Strategy.xlsx',engine='xlsxwriter')
hqm_dataframe_top50.to_excel(writer,sheet_name='momemtum strategy',index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [87]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [88]:
column_format = {
    'A':['ticker',string_template],
    'B':['price',dollar_template],
   'C' :['number of shares to buy',integer_template],
   'D': ['one-year price return',percent_template],
    'E':['one-year return percentile',percent_template],
   'F' :['six-month price return',percent_template],
    'G':['six-month return percentile',percent_template],
   'H' :['three-month price return',percent_template],
    'I':['three-month return percentile',percent_template],
    'J':['one-month price return',percent_template],
    'K':['one-month return percentile',percent_template],
    'L':['HQM score',percent_template],

    
}
    
for column in column_format.keys():
    writer.sheets['momemtum strategy'].set_column(f'{column}:{column}',22,column_format[column][1])

writer.save()
    


## Saving Our Excel Output

As before, saving our Excel output is very easy: